In [25]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup ,NavigableString, Comment
import html5lib

In [341]:
def get_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,  'html.parser')
    article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find_all('p', recursive = False)
    h2 = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find('h2')
    h2_text = '\n'.join(line.strip() for line in h2.get_text().split('\n') if line.strip())
    # Update the text of the h2 tag
    h2.string = h2_text
    for child in article[:-4]:
        imgs = child.find_all('img',recursive=True)
        print(len(imgs))
        a_tags = child.find_all('a')
        if len(a_tags) > 0:
            for a_tag in a_tags:
                try:
                    a_text = a_tag.text
                    parent_a_tag =  a_tag.parent
                    parent_a_tag.string = parent_a_tag.text
                except AttributeError:
                    break
            
        if len(imgs) >0:
            for t in imgs:
                try:
                    #t is image element
                    p_next = t.parent.next_sibling.next_sibling
                    #p_next.decompose()
                    #p_next['style'] = "text-align: center;"
                    caption_tag = soup.new_tag('caption')
                    t.insert_before(caption_tag)
                    caption_tag.append(t)
                    caption_tag['width'] = 800
                    caption_tag['align'] ="aligncenter"
                    caption_tag.append(p_next.text)
                    try:
                        t.attrs.pop('class')
                        t.attrs.pop('onclick')
                        t.attrs.pop('width')
                        t['src'] = t.get('data-original', None)
                        t['width'] = 800
                        t['height'] = 400
                    except KeyError:
                        break
                except AttributeError:
                    print(t)
    soup2 = BeautifulSoup("", 'html.parser')
    soup2.append(h2)
    for i in article[:-4]:
        if 'class' not in i.attrs:
            soup2.append(i)
        if i.string == "":
            i.decompse
        
    content = soup2.prettify()
    return content



In [148]:
#soup2 = BeautifulSoup("", 'html.parser')

def get_content2(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,  features='html')

    #remove the last 4 element
    article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
    
    children = article.find_all(recursive=False)
    for i in children[-4:]:
        i.decompose()
    caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
    #find a,span and remove tag with text
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    # set attribut for img
    b = article.find_all('img')
    for i in b:
        if '.svg' in i['src']:
            i.decompose()
    for i in article.find_all('img'):
        if 'data-original' in i.attrs.keys():
            i['src'] = i.get('data-original')
        #i.attrs = ['class', 'alt', 'src', 'data-original']
        del i['onclick']
        del i['style']
        del i['class']
    img_list = article.find_all('img')
    n_img = len(img_list)
    #print(len(caption_text_list))
    for i in range(0,n_img):
        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        caption_text = NavigableString(caption_text_list[i].string)
        caption_end = NavigableString("[/caption]")
        caption_text_list[i].decompose()
        # Insert the custom tags and caption text around the <img> tag
        img_list[i].insert_before(caption_start)
        img_list[i].insert_after(caption_end)
        try:
            img_list[i].insert_after(caption_text) 
        except IndexError as e:
            print(e)
        
        """caption_tag = soup.new_tag('caption')
        img_list[i].insert_before(caption_tag)
        caption_tag.append(img_list[i])
        caption_tag['width'] = 800
        caption_tag['align'] ="aligncenter"
        try:
            caption_tag.append(caption_text_list[i].string)
        except IndexError as e:
            print(e)
            #create caption"""
            
        #print(i.attrs.keys())"""
    #remove js  and css
    h2 = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find('h2')
    h2_text = '\n'.join(line.strip() for line in h2.get_text().split('\n') if line.strip())
    # Update the text of the h2 tag
    h2.string = h2_text
    #soup2 = BeautifulSoup("", 'html.parser')
    #soup2.append(h2)
    for script_or_style in article(['script', 'style']):
        script_or_style.decompose()
    for i in article.find_all(recursive = True):
        try:
            del i['onclick']
            del i['id']
            del i['class']
            del i['style']
        except AttributeError:
            continue
        try:
            print(i)
        except TypeError:
            continue
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
    for video in article.find_all('video'):
        video.decompose()
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: 24h.com.vn"  # Set the content of <i> tag
# Append the <i> tag as the last child of the <article> tag
    article.append(source_tag)
    for i in article.find_all('div', {'align': 'center'}):
        i.decompose()
    for element in article.find_all(recursive = True,string=True):
        if isinstance(element, NavigableString) and element.strip() == '':
            element.extract()
    for i in article.find_all('div'):
        if i.children == None and i.string == None:
            i.decompose()
    return article

In [149]:
test = get_content2('https://www.24h.com.vn/ban-tre-cuoc-song/cau-be-lop-9-truong-lang-duoc-tuyen-thang-vao-dai-hoc-so-1-chau-a-co-tri-nho-sieu-pham-kha-nang-tu-duy-logic-tuyet-dinh-c64a1521211.html')

<h2>Thành tích học tập của cậu bé này đã khiến không chỉ bạn bè cùng trang lứa mà thậm chí những người tiền bối đã học cao hơn cậu cũng phải ngưỡng mộ.</h2>
<p>Cậu bé 14 tuổi Quan Tử Khâm hiện đang là học sinh lớp 9 tại một trường trung học ở huyện Bác Bạch, tỉnh Quảng Tây, phía Nam Trung Quốc. Cuối năm 2022, Quan Tử Khâm đã vinh dự lọt vào danh sách trúng tuyển của Chương trình đào tạo tài năng hàng đầu về khoa học toán học của Đại học Thanh Hoa danh giá. </p>
<p>Điều này có nghĩa là cậu bé lớp 9 sẽ "nhảy cấp". Đây được xem là thành tích chưa từng có tiền lệ trước đó. Đã từng có trường hợp những học sinh lớp 1,2 nhảy cấp lên lớp 4,6, nhưng việc nhảy cấp từ trung học lên thẳng Đại học và đặc biệt là Thanh Hoa là tỷ lệ rất hiếm thấy. </p>
<p align="center"></p>
<div>
<div style="position:relative;max-width: 740px;margin: 0 auto">
<div><section class="txtCent bnrPtn inImgExpandSection" id="vnapzJKUhp_213_15s_container"><div id="vnapzJKUhp_213_15s">
</div>
</section>[caption id="" align="

In [60]:
response = requests.get('https://www.24h.com.vn/cong-nghe-thong-tin/apple-gay-soc-sap-cho-nguoi-dung-android-giao-tiep-voi-imessage-c55a1519650.html')
soup = BeautifulSoup(response.content, 'html.parser')
#remove the last 4 element
article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
children = article.find_all(recursive=False)
for i in children[-4:]:
    i.decompose()
#find a,span and remove tag with text
tags_to_remove = article.find_all(['a', 'span'])
for tag in tags_to_remove:
    # Extract the text from the tag
    tag_text = tag.get_text()
    # Replace the tag with its text content
    tag.replace_with(tag_text)
    tag.text.strip()
# set attribut for img
b = article.find_all('img')
for i in b:
    if '.svg' in i['src']:
        i.decompose()
for i in article.find_all('img'):
    if 'data-original' in i.attrs.keys():
        i['src'] = i.get('data-original')
    #i.attrs = ['class', 'alt', 'src', 'data-original']
    del i['onclick']
    del i['style']
    del i['class']
    i['width'] = 800
    i['height'] = 400

img_list = article.find_all('img')
n_img = len(img_list)
#print(n_img)
#print(len(caption_text_list))
for i in range(0,n_img):
        #create caption
        caption_tag = soup.new_tag('caption')
        img_list[i].insert_before(caption_tag)
        caption_tag.append(img_list[i])
        caption_tag['width'] = 800
        caption_tag['align'] ="aligncenter"
        caption_tag.append(caption_text_list[i])
    #print(i.attrs.keys())
#remove js  and css
h2 = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find('h2')
soup2 = BeautifulSoup("", 'html.parser')
soup2.append(h2)
for script_or_style in article(['script', 'style']):
    script_or_style.decompose()
for i in article.find_all(recursive = True):
    try:
        del i['id']
        del i['class']
    except AttributeError:
        continue
    try:
        print(i)
        soup2.append(i)
    except TypeError:
        continue
        
#for i in article.find_all('p',class_='img_chu_thich_0407'):
    #i.decompose()                


<p>Sau nhiều năm chịu sức ép bởi Google nhằm buộc Apple phải hỗ trợ giao thức tin nhắn RCS trong iMessage, nhà sản xuất iPhone cuối cùng cũng phải chấp nhận khi cam kết thực hiện điều này vào năm 2024.</p>
<p style="text-align: center;"><div id="container-24h-banner-in-image" style="text-align: center;width: 100%;clear: both;position: relative;">
<div style="position:relative;max-width: 740px;margin: 0 auto">
<div><section class="txtCent bnrPtn inImgExpandSection" id="JOyNWkLerd_213_15s_container"><div id="JOyNWkLerd_213_15s">
</div>
</section><caption align="aligncenter" width="800"><img alt="Apple gây sốc, sắp cho người dùng Android “giao tiếp” với iMessage - 1" height="400" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-17/2-1700203824-539-width740height416.jpg" width="800"/><p class="img_chu_thich_0407" style="text-align: center;">RCS cuối cùng cũng sắp có khả năng giao tiếp với iMessage.</p></caption></div>
<div id="24h-banner-in-image" style="left: 50%;transform: t

In [168]:
for i in article.find_all(recursive = False):
    try:
        print(i)
    except TypeError:
        continue


In [22]:
response = requests.get('https://www.24h.com.vn/cong-nghe-thong-tin/apple-gay-soc-sap-cho-nguoi-dung-android-giao-tiep-voi-imessage-c55a1519650.html')
soup = BeautifulSoup(response.content, 'html.parser')
#remove the last 4 element
article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')


In [23]:
def is_element_empty(element):
    """Check if a BeautifulSoup element is empty."""
    if isinstance(element, NavigableString):
        return element.strip() == ''
    if isinstance(element, Comment):
        return True  # Treat comments as empty

In [33]:
html_content = "<div><p>   </p><div></div><!-- Comment --></div>"
soup = BeautifulSoup(html_content, 'html.parser')

div_element = soup.find('div')
if is_element_empty(div_element):
    print("The element is empty")
else:
    print("The element is not empty")

The element is not empty


In [34]:

def is_only_whitespace(element):
    """Check if an element contains only whitespace or is empty."""
    if isinstance(element, NavigableString):
        return element.strip() == ''
    return all(is_only_whitespace(child) for child in element.contents)

def remove_whitespace_elements(soup):
    """Recursively remove all elements that contain only whitespace."""
    for element in soup.find_all():
        if is_only_whitespace(element):
            element.decompose()

# Example HTML content
html_content = """
<div>
    <p>   </p>
    <div>
        <span>    </span>
    </div>
    <p>Some text</p>
</div>
"""

soup = BeautifulSoup(html_content, 'html.parser')

# Remove elements that contain only whitespace
remove_whitespace_elements(soup)

# Print the modified HTML
print(soup.prettify())


<div>
 <p>
  Some text
 </p>
</div>



In [28]:
for i in article.find_all():
    print(i)

<h2 class="cate-24h-foot-arti-deta-sum ctTp tuht_show" id="article_sapo">
<strong>
                    Mặc dù chấp nhận hỗ trợ nhưng tin nhắn RCS từ người dùng Android vẫn chỉ là màu xanh lục như SMS thông thường.                </strong>
</h2>
<strong>
                    Mặc dù chấp nhận hỗ trợ nhưng tin nhắn RCS từ người dùng Android vẫn chỉ là màu xanh lục như SMS thông thường.                </strong>
<script type="text/javascript">window.onload = function () {resizeNewsImage("news-image", 740);}</script>
<p>Sau nhiều năm chịu sức ép bởi Google nhằm buộc Apple phải hỗ trợ giao thức tin nhắn RCS trong iMessage, nhà sản xuất iPhone cuối cùng cũng phải chấp nhận khi cam kết thực hiện điều này vào năm 2024.</p>
<p style="text-align: center;"></p>
<div id="container-24h-banner-in-image" style="text-align: center;width: 100%;clear: both;position: relative;">
<div style="position:relative;max-width: 740px;margin: 0 auto">
<div><section class="txtCent bnrPtn inImgExpandSection" id="aKyKRN

In [166]:

for i in article.find_all(recursive = True):
    print(i.attrs)

{}
{}
{}
{'style': 'text-align: center;'}
{'style': 'text-align: center;width: 100%;clear: both;position: relative;'}
{'style': 'position:relative;max-width: 740px;margin: 0 auto'}
{}
{}
{}
{'width': 800, 'align': 'aligncenter'}
{'alt': 'Apple gây sốc, sắp cho người dùng Android “giao tiếp” với iMessage - 1', 'src': 'https://image-us.24h.com.vn/upload/4-2023/images/2023-11-17/2-1700203824-539-width740height416.jpg', 'width': 800, 'height': 400}
None
{'id': '24h-banner-in-image', 'style': 'left: 50%;transform: translateX(-50%);position: absolute;display: block;height: 90px;color: white;bottom: 0px'}
{'id': 'RHOuJRPdEb_139_15s', 'class': ['txtCent']}
{'class': ['text_adver_right'], 'onclick': 'in_image_close();'}
{'id': '24h-banner-in-image-close', 'style': 'top:2px;right: 0px;position: absolute;display: none;overflow: hidden;width: 22px;height: 22px;color: #ccc;font-size: large;'}
{}
{}
{'style': 'text-align: center;'}
{'width': 800, 'align': 'aligncenter'}
{'alt': 'Apple gây sốc, sắp c

In [259]:
response = requests.get('https://www.24h.com.vn/cong-nghe-thong-tin/tri-tue-nhan-tao-lan-dau-duoc-ung-dung-vao-phong-luyen-thi-ao-nay-c55a1520038.html')
soup = BeautifulSoup(response.content,  features='html')
article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')


In [260]:
a = article.find_all(recursive=False)[-4:]
for i in a:
    i.decompose()

In [261]:
#article.find_all(recursive=False)[:-4]

In [262]:
len(article.find_all('img'))

2

In [263]:
for i in article.find_all('img'):
    if 'svg' in i['src']:
        i.decompose()
        print(len(article.find_all('img')))

1


In [264]:
children = article.find_all(recursive=False)
for i in children[-4:]:
    i.decompose()
article.find_all('img')

[<img alt="Trí tuệ nhân tạo lần đầu được ứng dụng vào phòng luyện thi ảo này - 1" class="news-image" onclick="show_slide_image_news(0)" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-19/giao-di---n-ph---n-ch---m-ch---a-speaking-1700360798-329-width740height539.png" style="width: 740px; height: 539px;"/>]

In [265]:
for i in children[-4:]:
    i.decompose()
for i in article.find_all('img'):
    if 'data-original' in i.attrs.keys():
        i['src'] = i.get('data-original')
    if 'svg' in i['src']:
        i.decompose()
    #i.attrs = ['class', 'alt', 'src', 'data-original']
    #del i['onclick']
    del i['style']
    del i['class']
    i['width'] = 800
    i['height'] = 400
    n_img = len(article.find_all('img'))
    img_list = article.find_all('img')
    caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
    for i in range(0,n_img):
        #create caption
        caption_tag = soup.new_tag('caption')
        img_list[i].insert_before(caption_tag)
        caption_tag.append(img_list[i])
        caption_tag['width'] = 800
        caption_tag['align'] ="aligncenter"
        #caption_tag.append(caption_text_list[i])                
    #print(i.attrs.keys())

In [266]:
caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
caption_text_list

[<p class="img_chu_thich_0407" style="text-align: center;">Giao diện phần chấm chữa Speaking của Prep.</p>]

In [256]:
article.find_all('img')

[<img alt="Luật sư Đỗ Văn Luận: “Thầy giáo dạy Lịch sử là người đã sinh ra tôi lần thứ hai” - 1" height="400" onclick="show_slide_image_news(0)" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-17/picture-1-1700205769-368-width1178height1600.jpg" width="800"/>,
 <img alt="Luật sư Đỗ Văn Luận: “Thầy giáo dạy Lịch sử là người đã sinh ra tôi lần thứ hai” - 2" data-original="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-17/picture-2-1700205769-343-width1600height1066.jpg" height="400" onclick="show_slide_image_news(1)" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-17/picture-2-1700205769-343-width1600height1066.jpg" width="800"/>]

In [57]:
for i in article.find_all('img'):
    print(i)

<img alt="Apple gây sốc, sắp cho người dùng Android “giao tiếp” với iMessage - 1" height="400" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-17/2-1700203824-539-width740height416.jpg" width="800"/>
<img alt="Apple gây sốc, sắp cho người dùng Android “giao tiếp” với iMessage - 2" class="news-image" data-original="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-17/3-1700203830-742-width740height493.jpg" onclick="show_slide_image_news(1)" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABAQMAAAAl21bKAAAAA1BMVEX6+vqsEtnpAAAACklEQVQI12NgAAAAAgAB4iG8MwAAAABJRU5ErkJggg==" style="width: 740px; height: 493px;"/>


In [58]:
len(article.find_all('img'))

2

In [52]:
for i in children[-4:]:
    i.decompose()

In [54]:
article.find_all('p',class_='img_chu_thich_0407')

[<p class="img_chu_thich_0407" style="text-align: center;">RCS cuối cùng cũng sắp có khả năng giao tiếp với iMessage.</p>,
 <p class="img_chu_thich_0407" style="text-align: center;">Người dùng iMessage ở Mỹ rất kỳ thị tin nhắn SMS.</p>]

In [3]:


# URL of the webpage to scrape
url = 'https://www.24h.com.vn/cong-nghe-thong-tin/apple-gay-soc-sap-cho-nguoi-dung-android-giao-tiep-voi-imessage-c55a1519650.html'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove <script> and <style> elements
    for script_or_style in soup(['script', 'style']):
        script_or_style.decompose()


    # Now, soup contains the cleaned HTML
    # Extract the required data from soup
    # ...
    article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
    print(article)

else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


<article class="cate-24h-foot-arti-deta-info">
<h2 class="cate-24h-foot-arti-deta-sum ctTp tuht_show" id="article_sapo">
<strong>
                    Mặc dù chấp nhận hỗ trợ nhưng tin nhắn RCS từ người dùng Android vẫn chỉ là màu xanh lục như SMS thông thường.                </strong>
</h2>
 <p>Sau nhiều năm chịu sức ép bởi Google nhằm buộc <a class="TextlinkBaiviet" href="https://www.24h.com.vn/apple-c55e3987.html" title="Apple">Apple</a> phải hỗ trợ giao thức tin nhắn RCS trong iMessage, nhà sản xuất <a class="TextlinkBaiviet" href="https://www.24h.com.vn/iphone-c407e1576.html" title="iPhone">iPhone</a> cuối cùng cũng phải chấp nhận khi cam kết thực hiện điều này vào năm 2024.</p>
<p style="text-align: center;"><div id="container-24h-banner-in-image" style="text-align: center;width: 100%;clear: both;position: relative;">
<div style="position:relative;max-width: 740px;margin: 0 auto">
<div><section class="txtCent bnrPtn inImgExpandSection" id="hHFQbUSLCF_213_15s_container"><div id="hH

In [368]:
soup.find('time').text.strip()

AttributeError: 'NoneType' object has no attribute 'text'

In [261]:
def conver_time_string(posted_date):
    pattern = r'\d{2}/\d{2}/\d{4}'
    match = re.search(pattern, posted_date)

    if match:
        date_string = match.group()

        # Convert to datetime object
        datetime_obj = datetime.strptime(date_string, "%d/%m/%Y")
        return datetime_obj
    else:
        return ''

In [262]:
conver_time_string(soup.find('time').text.strip())

datetime.datetime(2023, 11, 18, 0, 0)

'2023-11-21'

In [288]:
def convert_string(input_str):
    # Extract the date part using regular expression
    pattern = r'\d{2}/\d{2}/\d{4}'
    date_part = re.search(pattern, input_str).group()
    match = re.search(pattern, input_str)
    if match:
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_part, '%d/%m/%Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    else:
        return ""
    

In [284]:
# Format the datetime object intoZZZZ the desired string format
=ZZ+

In [2]:
# Example HTML content
html_content = """
<div>
    <p>This is a <a href="http://example.com">link</a> in a paragraph.</p>
</div>
"""

soup = BeautifulSoup(html_content, 'html.parser')

# Find the <a> tag within the <div>
a_tag = soup.find('div').find('a')

# Extract the text from the <a> tag
a_text = a_tag.get_text()

# Replace the <a> tag with its text content
a_tag.replace_with(a_text)

# Print the modified HTML
print(soup.prettify())

<div>
 <p>
  This is a
  link
  in a paragraph.
 </p>
</div>



In [234]:
def send_post_to_5goals(title,content,category_id,published_date)
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "token": "5goalvodichcmnl",  # Replace with your actual access token
        "published_date": published_date,
        "category_id": category_id  # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")

The post was successfully created.
Response: {"success":true,"post_id":62860}


In [ ]:
def main():
    web_24h_com_vn = get_news()
    for i in list(web_24h_com_vn2['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(web_24h_com_vn2['urls'][i]['sub-category']):
            url_list =  web_24h_com_vn2['urls'][i]['sub-category'][j]['url_list']
            for t in range(0,len(url_list)):
                text = web_24h_com_vn2['urls'][i]['sub-category'][j]['content'][t]['text']
                title = web_24h_com_vn2['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = web_24h_com_vn2['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = web_24h_com_vn2['urls'][i]['cate_id']
                send_post_to_5goals(title,text,cate_id,published_date)